# Preamble

In [ ]:
strain='rdv2'
process = 'wholepop'
programs = ['aas', 'ras', 'dta', 'atd', 'dv-mono', 'dv-pert', 'dv-comp']

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from scipy.interpolate import interp1d
import matplotlib

%matplotlib inline
plt.viridis()

import sys
sys.path.extend(["../../lsc"])
import lsc
import plotting_resources as pr

data_path = "../cph8-ompr_data"

white = 159.4

dfs = {}
for pt in programs:
    prog_path = os.path.join(data_path, pt)
    xl_path = os.path.join(prog_path, '{}_{}_{}_complete.xlsx'.format(strain,process,pt))
    
    df = pd.read_excel(xl_path)
    df['FL1 Mean'] = df['FL1 Mean'] - white
    
    if pt == 'dv-mono':
        dfs['dv-mono-r'] = df.loc[df['Bot LED ID'] == '660-LS']
        dfs['dv-mono-uv'] = df.loc[df['Bot LED ID'] == '380-SB']
    else:
        dfs[pt] = df
        
# Cleaning data
eliminated_samples = []

pts = ['dv-pert', 'dv-comp', 'dv-mono-r', 'dv-mono-uv']
for i in range(97,193):
    pt = pts[(i-97)/24]
    es1 = (pt, 'rdv2_r05_s{:03d}'.format(i))
    es2 = (pt, 'rdv2_r06_s{:03d}'.format(i))
    eliminated_samples.append(es1)
    eliminated_samples.append(es2)
    
for es in eliminated_samples:
    df = dfs[es[0]]
    idx = df.loc[df['Sample ID'] == es[1]].index.tolist()[0]
    df.at[idx, 'FL1 Mean'] = np.nan
    
prog_signals_path = "../../gene_expression_programming_signals"
ref_path = os.path.join(prog_signals_path, 'programming_reference_signals.xlsx')
ref_df = pd.read_excel(ref_path)

genprog_path = os.path.join(prog_signals_path, 'generated_light_programs.xls')
genprog_df = pd.read_excel(genprog_path)
        
## Set the default plot output to be inline in this notebook
%matplotlib inline

## Initialize plotting style
pr.init_style()

## Set plotting output directory

out_folder = './plots/'

## Set default axes ranges, ticks, and labels for common axes
# Time
t_range = [0,360]
t_ticks = [0,120,240,360]
t_label = "Time (min)"

# GFP fluorescence
gfp_range = [0,5.5e3]
gfp_ticks = [0,1e3,2e3,3e3,4e3]
gfp_range_log = [3e1, 2e4]
gfp_ticks_log = [1e2, 1e3, 1e4]
gfp_label = "GFP fluorescence (MEFL)"

# Light intensity
int_range_lin = [0, 20]
int_range_log = [20 / 2**11, 20]
int_ticks_lin = [0, 10, 20]
int_ticks_log = [1e-2,1e-1,1e0,1e1]
int_label = "Light intensity (\si{\micro\mole.m^{-2}.s^{-1}})"

## Set up color cycles
green_cycle = pr.red_cycle
led_cycle = pr.led_colors

# Dynamic data

## Simulate dynamic data using fitted model

In [ ]:
df = dfs['dta'].sort_values(['Activating intensity', 'Data time point'])
X = np.reshape(df['Data time point'], (8,24))
Y = np.reshape(df['FL1 Mean'], (8,24))
intens = np.reshape(df['Activating intensity'], (8,24))[:,1]

In [ ]:
reload(lsc)
m = lsc.Model.from_path('../cph8-ompr_model_parameters/rdv2_m01/all_data_onetau_with_kholds_and_adjusted_initial.params', hill_type='rep')
m.params['b'].value = m.params['b'].value - white

dta_progs = [lsc.LEDProgram.fromdict({'id': '660-ls', 'pre': 0, 't': [0], 'int': [i]}) for i in intens]
atd_progs = [lsc.LEDProgram.fromdict({'id': '660-ls', 'pre': i, 't': [0], 'int': [0]}) for i in intens]

dta_mdl_funcs = [m.simulate_led(dta_progs[i]) for i in range(len(dta_progs))]
atd_mdl_funcs = [m.simulate_led(atd_progs[i]) for i in range(len(atd_progs))]

progs = {'dta': dta_progs, 'atd': atd_progs}
sims = {'dta': dta_mdl_funcs, 'atd': atd_mdl_funcs}

## Plot dynamic data (gfp vs. time)

In [ ]:
datasets = ['dta', 'atd']
scales = ['lin','logy','logxy']
models = ['raw', 'model']

# styling the plot
pr.init_style()

plt.rcParams['axes.color_cycle'] = green_cycle
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['legend.loc'] = 'upper center'
plt.rcParams['legend.fontsize'] = plt.rcParams['font.size'] * 0.6
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handletextpad'] = 0.3
plt.rcParams['legend.borderaxespad'] = 0.5

for dset in datasets:
    for model in models: # plots the raw and actual model
        df = dfs[dset[0:3]].sort_values(['Activating intensity', 'Data time point']) # I these next few lines create new arrays with modified dimensions
        X = np.reshape(df['Data time point'], (8,24))
        Y = np.reshape(df['FL1 Mean'], (8,24))
        intens = np.reshape(df['Activating intensity'], (8,24))[:,1]

        for scale in scales: # plots the model for each of the three scales (lin, logy, and logxy)

            if model in ['raw']:
                for i in range(len(intens)):
                    plt.plot(X[i], Y[i], '.-', label='{:.2f}'.format(intens[i]))
                    
            if model in ['model']:
                if scale in ['lin']:
                    Xm = np.linspace(0,360,361)
                if scale in ['logx', 'logxy']:
                    Xm = np.logspace(np.log10(5), np.log10(360), 361)
                    
                for i in range(len(intens)):
                    Ym = sims[dset][i](Xm)
                    plt.plot(Xm, Ym, '-', label=None)

                for i in range(len(intens)):
                    plt.plot(X[i], Y[i], '.', label='{:.2f}'.format(intens[i]))
            if scale == 'lin':                
                plt.xlim(t_range)
                plt.xlabel(t_label)
                plt.xticks(t_ticks)

                plt.ylim(gfp_range)
                plt.ylabel(gfp_label)
                plt.yticks(gfp_ticks)

                plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

                ax = plt.gca()
                ax.minorticks_on()
                ax.tick_params(axis='x',which='both',top='off')
                ax.tick_params(axis='y',which='both',right='off')
#                 ax.set_aspect(1./ax.get_data_ratio())
#                 plt.tight_layout()

            if scale == 'logy':
                plt.xlim(t_range)
                plt.xlabel(t_label)
                plt.xticks(t_ticks)

                plt.yscale('log')
                plt.ylim(gfp_range_log)
                plt.ylabel(gfp_label)
                plt.yticks(gfp_ticks_log)

    #             plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

                ax = plt.gca()
                ax.minorticks_on()
                ax.tick_params(axis='x',which='both',top='off')
                ax.tick_params(axis='y',which='both',right='off')
    #             ax.set_aspect(1./ax.get_data_ratio())
#                 plt.tight_layout()

            if scale == 'logxy':
                plt.xscale('log')
                plt.xlim(t_range)
                plt.xlabel(t_label)
    #             plt.xticks(t_ticks)

                plt.yscale('log')
                plt.ylim(gfp_range_log)
                plt.ylabel(gfp_label)
                plt.yticks(gfp_ticks_log)

    #             plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

                ax = plt.gca()
                ax.minorticks_on()
                ax.tick_params(axis='x',which='both',top='off')
                ax.tick_params(axis='y',which='both',right='off')
    #             ax.set_aspect(1./ax.get_data_ratio())
            
            plt.tight_layout()
    
            plt.legend(mode = 'expand', ncol = 4, title=int_label)
            ax = plt.gca()
            ax.get_legend().get_title().set_fontsize(plt.rcParams['font.size'] * 0.8)

            plot_name = '{}_{}_{}.png'.format(dset, scale, model)
            plt.savefig(out_folder + plot_name)
        #         plt.rcParams['savefig.dpi'] = 100
            plt.show()
            plt.close()

## Plot dynamic data (heatmap)

In [ ]:
datasets = ['dta', 'atd']
scales = ['lin','logz']
models = ['raw', 'model']

pr.init_style()

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as ticker
from matplotlib.colors import LogNorm

plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['axes.color_cycle'] = led_cycle
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['legend.loc'] = 'upper center'
plt.rcParams['legend.fontsize'] = plt.rcParams['font.size'] * 0.6
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handletextpad'] = 0.3
plt.rcParams['legend.borderaxespad'] = 0.5

def fmt(x, pos):
    a, b = '{:.0e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)

for model in models:
    for dset in datasets:
        for scale in scales:
            df = dfs[dset].sort_values(['Activating intensity', 'Data time point'])
            X = np.reshape(df['Data time point'], (8,24))
            Y = np.reshape(df['Activating intensity'], (8,24))
            if model in ['raw']:
                Z = np.reshape(df['FL1 Mean'], (8,24))

            elif model in ['model']:
                Xm = np.reshape(df['Data time point'], (8,24))
                Z = np.array([sims[dset][i](Xm[i]) for i in range(len(Y))])

            if scale == 'lin':          

                im = plt.imshow(np.flipud(Z), interpolation='nearest', vmin=0, vmax=4e3, cmap='viridis')

                ax = plt.gca()

                cbar = plt.colorbar(im,fraction=0.0242, pad=0.01, format=ticker.FuncFormatter(fmt))
                cbar.set_label(gfp_label)
                cbar.set_ticks(gfp_ticks)

            if scale == 'logz':
                im = plt.imshow(np.flipud(Z), interpolation='nearest', norm=LogNorm(vmin=3e1, vmax=4e3), cmap='viridis')

                ax = plt.gca()

                cbar = plt.colorbar(im,fraction=0.0242, pad=0.01, format=ticker.FuncFormatter(fmt))
                cbar.set_label(gfp_label)
                cbar.set_ticks(gfp_ticks_log)

            plt.xlabel("Time (min)")
            xt = [x for x in X[0,:]]
            ax.set_xticks(range(len(xt)))
            ax.set_xticklabels(['{:.0f}'.format(x) for x in xt],rotation=60)

            round_to_n = lambda x, n: np.round(x, -int(np.floor(np.log10(x))) + (n - 1)) 
            plt.ylabel(int_label)
            ax.set_yticks(range(len(Y[:,0])))
            ax.set_yticklabels(['{}'.format(round_to_n(y, 2)) for y  in Y[::-1,0]])

            ax = plt.gca()
            ax.minorticks_on()
            ax.tick_params(axis='x',which='both',top='off')
            ax.tick_params(axis='x',which='both',bottom='off')
            ax.tick_params(axis='y',which='both',left='off')
            ax.tick_params(axis='y',which='both',right='off')
            ax.set_aspect(0.5*1./ax.get_data_ratio())
            plt.tight_layout()

            plot_name = '{}_{}_{}_hmap.png'.format(dset, scale, model)
            plt.savefig(out_folder + plot_name)
    #         plt.rcParams['savefig.dpi'] = 100
            plt.show()
            plt.close()

## Plot dynamic data (heapmap, no labels)

In [ ]:
datasets = ['dta', 'atd']
scales = ['logz']
models = ['raw', 'model']

pr.init_style()

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as ticker
from matplotlib.colors import LogNorm

plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['axes.color_cycle'] = led_cycle
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['legend.loc'] = 'upper center'
plt.rcParams['legend.fontsize'] = plt.rcParams['font.size'] * 0.6
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handletextpad'] = 0.3
plt.rcParams['legend.borderaxespad'] = 0.5

def fmt(x, pos):
    a, b = '{:.0e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)

for model in models:
    for dset in datasets:
        for scale in scales:
            
            if model in ['raw']:
                df = dfs[dset].sort_values(['Activating intensity', 'Data time point'])
                X = np.reshape(df['Data time point'], (8,24))
                Y = np.reshape(df['Activating intensity'], (8,24))
                Z = np.reshape(df['FL1 Mean'], (8,24))

            elif model in ['model']:
                Xm = np.reshape(df['Data time point'], (8,24))
                Z = np.array([sims[dset][i](Xm[i]) for i in range(len(Y))])

            if scale == 'lin':          

                im = plt.imshow(np.flipud(Z), interpolation='nearest', vmin=0, vmax=1.2e5, cmap='viridis')

                ax = plt.gca()

                cbar = plt.colorbar(im,fraction=0.0242, pad=0.01, format=ticker.FuncFormatter(fmt))
                cbar.set_label(gfp_label)
                cbar.set_ticks([0,3e4,6e4,9e4,1.2e5])

            if scale == 'logz':
                im = plt.imshow(np.flipud(Z), interpolation='nearest', norm=LogNorm(vmin=3e1, vmax=4e3), cmap='viridis')

                ax = plt.gca()

#                 cbar = plt.colorbar(im,fraction=0.0242, pad=0.01, format=ticker.FuncFormatter(fmt))
#                 cbar.set_label(gfp_label)
#                 cbar.set_ticks([1e3, 3e3, 1e4, 3e4, 1e5])

#             plt.xlabel("Time (min)")
#             xt = [x for x in X[0,:]]
#             ax.set_xticks(range(len(xt)))
#             ax.set_xticklabels(['{:.0f}'.format(x) for x in xt],rotation=60)
            ax.set_xticks([])
            ax.set_yticks([])

#             round_to_n = lambda x, n: np.round(x, -int(np.floor(np.log10(x))) + (n - 1)) 
#             plt.ylabel(int_label)
#             ax.set_yticks(range(len(Y[:,0])))
#             ax.set_yticklabels(['{}'.format(round_to_n(y, 2)) for y  in Y[::-1,0]])

            ax = plt.gca()
            ax.minorticks_on()
            ax.tick_params(axis='x',which='both',top='off')
            ax.tick_params(axis='x',which='both',bottom='off')
            ax.tick_params(axis='y',which='both',left='off')
            ax.tick_params(axis='y',which='both',right='off')
            ax.set_aspect(0.33*1./ax.get_data_ratio())
#             plt.tight_layout()

            plot_name = '{}_{}_{}_nolabel_hmap.png'.format(dset, scale, model)
            plt.savefig(out_folder + plot_name, pad_inches=0, bbox_inches='tight')
    #         plt.rcParams['savefig.dpi'] = 100
            plt.show()
            plt.close()

## Generate colorbar

In [ ]:
cbar = plt.colorbar(im,fraction=0.0242, pad=0.01, format=ticker.FuncFormatter(fmt))
cbar.set_ticks([3e1, 1e2, 3e2, 1e3, 3e3])
cbar.set_ticklabels([])
plot_name = 'cbar.png'.format(dset, scale, model)
plt.savefig(out_folder + plot_name, bbox_inches='tight', pad_inches=0.1)

## Plot dynamic data residuals

In [ ]:
datasets = ['dta', 'atd']
scales = ['abs','rel']
models = ['residual']

pr.init_style()

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as ticker
from matplotlib.colors import LogNorm

plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['axes.color_cycle'] = led_cycle
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['legend.loc'] = 'upper center'
plt.rcParams['legend.fontsize'] = plt.rcParams['font.size'] * 0.6
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handletextpad'] = 0.3
plt.rcParams['legend.borderaxespad'] = 0.5

def fmt(x, pos):
    a, b = '{:.0e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)

for model in models:
    for dset in datasets:
        for scale in scales:
            
            df = dfs[dset].sort_values(['Activating intensity', 'Data time point'])
            Xd = np.reshape(df['Data time point'], (8,24))
            Yd = np.reshape(df['Activating intensity'], (8,24))
            Zd = np.reshape(df['FL1 Mean'], (8,24))

            Xm = np.reshape(df['Data time point'], (8,24))
            Zm = np.array([sims[dset][i](Xm[i]) for i in range(len(Y))])
            
            if scale == 'abs':
                
                Z = Zd - Zm

                im = plt.imshow(np.flipud(Z), interpolation='nearest', vmin=-4e2, vmax=4e2, cmap='bwr')

                ax = plt.gca()

                cbar = plt.colorbar(im,fraction=0.0242, pad=0.01, format=ticker.FuncFormatter(fmt))
                cbar.set_label('Absolute error (MEFL)')
                cbar.set_ticks([-4e2, 0, 4e2])

            if scale == 'rel':
                
                Z = np.log2(Zd/Zm)
                
                im = plt.imshow(np.flipud(Z), interpolation='nearest', vmin=-1, vmax=1, cmap='bwr')

                ax = plt.gca()

                cbar = plt.colorbar(im,fraction=0.0242, pad=0.01, format=ticker.FuncFormatter(fmt))
                cbar.set_label('Log$_{2}$ relative error')
                cbar.set_ticks([-1, 0, 1])

            plt.xlabel("Time (min)")
            xt = [x for x in X[0,:]]
            ax.set_xticks(range(len(xt)))
            ax.set_xticklabels(['{:.0f}'.format(x) for x in xt],rotation=60)

            round_to_n = lambda x, n: np.round(x, -int(np.floor(np.log10(x))) + (n - 1)) 
            plt.ylabel(int_label)
            ax.set_yticks(range(len(Yd[:,0])))
            ax.set_yticklabels(['{}'.format(round_to_n(y, 2)) for y  in Yd[::-1,0]])

            ax = plt.gca()
            ax.minorticks_on()
            ax.tick_params(axis='x',which='both',top='off')
            ax.tick_params(axis='x',which='both',bottom='off')
            ax.tick_params(axis='y',which='both',left='off')
            ax.tick_params(axis='y',which='both',right='off')
            ax.set_aspect(0.5*1./ax.get_data_ratio())
            plt.tight_layout()

            plot_name = '{}_{}_{}_hmap.png'.format(dset, scale, model)
            plt.savefig(out_folder + plot_name)
    #         plt.rcParams['savefig.dpi'] = 100
            plt.show()
            plt.close()

# Spectral data

## Simulate spectral data using fitted model

In [ ]:
df = dfs['aas'].sort_values(['Spectral intensity','Top Centroid (nm)'])
X = np.reshape(np.repeat(pr.led_centroids,12), (12,23))
Y = np.reshape(df['Spectral intensity'], (12,24))[:,:23]
Z = np.reshape(df['FL1 Mean'], (12,24))[:,:23]
Z[-3:,0] = np.nan

m = lsc.Model.from_path('../cph8-ompr_model_parameters/rdv2_m01/all_data_onetau_with_kholds_and_adjusted_initial.params', hill_type='rep')
m.params['b'].value = m.params['b'].value - white

aas_progs = [[0 for x in range(Z.shape[1])] for y in range(Z.shape[0])]
ras_progs = [[0 for x in range(Z.shape[1])] for y in range(Z.shape[0])]
led_names = pr.led_names
for i,x in enumerate(X[:,0]):
    for j,y in enumerate(Y[0,:]):
        l1 = lsc.LEDProgram(led_names[j].lower(), Y[i,0])
        aas_progs[i][j] = l1
        l2 = lsc.LEDProgram('660-ls', 1.25)
        ras_progs[i][j] = lsc.WellProgram.fromlist([l1, l2])
        
sims['aas'] = np.array([[m.simulate_led(aas_progs[i][j]) for i in range(len(aas_progs))] for j in range(len(aas_progs[0]))]).T
sims['ras'] = np.array([[m.simulate_well(ras_progs[i][j]) for i in range(len(ras_progs))] for j in range(len(ras_progs[0]))]).T

## Plot spectral data (heatmap)

In [ ]:
datasets = ['aas', 'ras']
scales = ['lin','logz']
models = ['raw', 'model']

pr.init_style()

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as ticker
from matplotlib.colors import LogNorm

plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['axes.color_cycle'] = led_cycle
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['legend.loc'] = 'upper center'
plt.rcParams['legend.fontsize'] = plt.rcParams['font.size'] * 0.6
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handletextpad'] = 0.3
plt.rcParams['legend.borderaxespad'] = 0.5

def fmt(x, pos):
    a, b = '{:.0e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)

for model in models:
    for dset in datasets:
        for scale in scales:
            df = dfs[dset].sort_values(['Spectral intensity','Top Centroid (nm)'])
            
            if dset in ['aas']:
                imax = 12
                plt.rcParams['figure.figsize'] = (8,4)
            if dset in ['ras']:
                imax = 4
                plt.rcParams['figure.figsize'] = (8,4)
            
            if model in ['raw']:
                X = np.reshape(np.repeat(pr.led_centroids,imax), (imax,23))
                Y = np.reshape(df['Spectral intensity'], (imax,24))[:imax,:23]
                A = np.reshape(df['Activating intensity'], (imax,24))[:imax,:23]
                Z = np.reshape(df['FL1 Mean'], (imax,24))[:,:23]
                Z[-3:,0] = np.nan
            
            if model in ['model']:
                imax = 12
                df = dfs['aas'].sort_values(['Spectral intensity','Top Centroid (nm)'])
                X = np.reshape(np.repeat(pr.led_centroids,imax), (imax,23))
                Y = np.reshape(df['Spectral intensity'], (imax,24))[:imax,:23]
                Z = sims[dset]
            
            if scale == 'lin':          
                im = plt.imshow(np.flipud(Z), interpolation='nearest', vmin=0, vmax=4e3, cmap='viridis')

                ax = plt.gca()

                cbar = plt.colorbar(im,fraction=0.0242, pad=0.01, format=ticker.FuncFormatter(fmt))
                cbar.set_label(gfp_label)
                cbar.set_ticks(gfp_ticks)
#                 ax.set_aspect(0.5*1./ax.get_data_ratio())

            if scale == 'logz':
                im = plt.imshow(np.flipud(Z), interpolation='nearest', norm=LogNorm(vmin=3e1, vmax=4e3), cmap='viridis')

                ax = plt.gca()

                cbar = plt.colorbar(im,fraction=0.0242, pad=0.01, format=ticker.FuncFormatter(fmt))
                cbar.set_label(gfp_label)
                cbar.set_ticks(gfp_ticks_log)
#                 ax.set_aspect(0.5*1./ax.get_data_ratio())

            plt.xlabel("LED centroid wavelength (nm)")
            ax.set_xticks([i for i in range(len(pr.led_centroids))])
            ax.set_xticklabels(['{:.0f}'.format(i) for i in pr.led_centroids],rotation=60)

            round_to_n = lambda x, n: np.round(x, -int(np.floor(np.log10(x))) + (n - 1)) 
            plt.ylabel(int_label)
            ax.set_yticks(range(len(Y[:,0])))
            ax.set_yticklabels(['{}'.format(round_to_n(y, 2)) for y  in Y[::-1,0]])
    #         plt.ylabel(int_label)
    #         inten_ticks = [10, 1.0, 0.1, 0.01]
    #         def scale_inten(inten):
    #             return 1-np.log2(inten/10.0)
    #         ax.set_yticks([scale_inten(i) for i in inten_ticks])
    #         intensity_labels = ['{}'.format(i) for i in inten_ticks]
    #         ax.set_yticklabels(intensity_labels)

            ax = plt.gca()
            ax.minorticks_on()
            ax.tick_params(axis='x',which='both',top='off')
            ax.tick_params(axis='x',which='both',bottom='off')
            ax.tick_params(axis='y',which='both',left='off')
            ax.tick_params(axis='y',which='both',right='off')

            plt.tight_layout()

            plot_name = '{}_{}_{}_heatmap.png'.format(dset, scale, model)
            plt.savefig(out_folder + plot_name)
    #         plt.rcParams['savefig.dpi'] = 100
            plt.show()
            plt.close()

## Plot spectral data (heatmap, no labels)

In [ ]:
datasets = ['aas', 'ras']
scales = ['logz']
models = ['raw', 'model']

pr.init_style()

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as ticker
from matplotlib.colors import LogNorm

plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['axes.color_cycle'] = led_cycle
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['legend.loc'] = 'upper center'
plt.rcParams['legend.fontsize'] = plt.rcParams['font.size'] * 0.6
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handletextpad'] = 0.3
plt.rcParams['legend.borderaxespad'] = 0.5

def fmt(x, pos):
    a, b = '{:.0e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)

for model in models:
    for dset in datasets:
        for scale in scales:
            
            if dset in ['aas']:
                imax = 12
            if dset in ['ras']:
                imax = 4
            
            df = dfs[dset].sort_values(['Spectral intensity','Top Centroid (nm)'])
            X = np.reshape(np.repeat(pr.led_centroids,12), (12,23))
            Y = np.reshape(df['Spectral intensity'], (imax,24))[:imax,:23]
            A = np.reshape(df['Activating intensity'], (imax,24))[:imax,:23]
            
            if model in ['raw']:
                Z = np.reshape(df['FL1 Mean'], (imax,24))[:imax,:23]
                Z[-3:,0] = np.nan
            
            if model in ['model']:
                Z = sims[dset]
            
            if scale == 'lin':          
                im = plt.imshow(np.flipud(Z), interpolation='nearest', vmin=0, vmax=1.2e5, cmap='viridis')

                ax = plt.gca()

                cbar = plt.colorbar(im,fraction=0.0242, pad=0.01, format=ticker.FuncFormatter(fmt))
                cbar.set_label(gfp_label)
                cbar.set_ticks([0,3e4,6e4,9e4,1.2e5])
                ax.set_aspect(0.5*1./ax.get_data_ratio())

            if scale == 'logz':
                im = plt.imshow(np.flipud(Z), interpolation='nearest', norm=LogNorm(vmin=3e1, vmax=4e3), cmap='viridis')

                ax = plt.gca()

#                 cbar = plt.colorbar(im,fraction=0.0242, pad=0.01, format=ticker.FuncFormatter(fmt))
#                 cbar.set_label(gfp_label)
#                 cbar.set_ticks([1e3, 3e3, 1e4, 3e4, 1e5])
                ax.set_aspect(0.5*1./ax.get_data_ratio())

#             plt.xlabel("LED centroid wavelength (nm)")
#             ax.set_xticks([i for i in range(len(pr.led_centroids))])
#             ax.set_xticklabels(['{:.0f}'.format(i) for i in pr.led_centroids],rotation=60)

#             round_to_n = lambda x, n: np.round(x, -int(np.floor(np.log10(x))) + (n - 1)) 
#             plt.ylabel(int_label)
#             ax.set_yticks(range(len(Y[:,0])))
#             ax.set_yticklabels(['{}'.format(round_to_n(y, 2)) for y  in Y[::-1,0]])
    #         plt.ylabel(int_label)
    #         inten_ticks = [10, 1.0, 0.1, 0.01]
    #         def scale_inten(inten):
    #             return 1-np.log2(inten/10.0)
    #         ax.set_yticks([scale_inten(i) for i in inten_ticks])
    #         intensity_labels = ['{}'.format(i) for i in inten_ticks]
    #         ax.set_yticklabels(intensity_labels)
    
            ax.set_xticks([])
            ax.set_yticks([])

            ax = plt.gca()
            ax.minorticks_on()
            ax.tick_params(axis='x',which='both',top='off')
            ax.tick_params(axis='x',which='both',bottom='off')
            ax.tick_params(axis='y',which='both',left='off')
            ax.tick_params(axis='y',which='both',right='off')

            plt.tight_layout()

            plot_name = '{}_{}_{}_nolabel_heatmap.png'.format(dset, scale, model)
            plt.savefig(out_folder + plot_name, bbox_inches='tight', pad_inches=0)
    #         plt.rcParams['savefig.dpi'] = 100
            plt.show()
            plt.close()

## Plot spectral residuals

In [ ]:
datasets = ['aas', 'ras']
scales = ['abs','rel']
models = ['residual']

pr.init_style()

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as ticker
from matplotlib.colors import LogNorm

plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['axes.color_cycle'] = led_cycle
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['legend.loc'] = 'upper center'
plt.rcParams['legend.fontsize'] = plt.rcParams['font.size'] * 0.6
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handletextpad'] = 0.3
plt.rcParams['legend.borderaxespad'] = 0.5

def fmt(x, pos):
    a, b = '{:.0e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)

for model in models:
    for dset in datasets:
        for scale in scales:
            
            
#             X = np.reshape(np.repeat(pr.led_centroids,imax), (imax,23))
#             Y = np.reshape(df['Spectral intensity'], (imax,24))[:imax,:23]
#             A = np.reshape(df['Activating intensity'], (imax,24))[:imax,:23]
#             Z = np.reshape(df['FL1 Mean'], (imax,24))[:,:23]
            
            if dset in ['aas']:
                imax = 12
                plt.rcParams['figure.figsize'] = (8,4)
            if dset in ['ras']:
                imax = 4
                plt.rcParams['figure.figsize'] = (8,4)
            
            df = dfs[dset].sort_values(['Spectral intensity','Top Centroid (nm)'])
            X = np.reshape(np.repeat(pr.led_centroids,imax), (imax,23))
            Y = np.reshape(df['Spectral intensity'], (imax,24))[:imax,:23]
            A = np.reshape(df['Activating intensity'], (imax,24))[:imax,:23]
            
            Zd = np.reshape(df['FL1 Mean'], (imax,24))[:imax,:23]
            Zd[-3:,0] = np.nan
            
            Zm =sims[dset][12-imax:,:23]
            
            if scale == 'abs':
                
                Z = Zd - Zm

                im = plt.imshow(np.flipud(Z), interpolation='nearest', vmin=-4e2, vmax=4e2, cmap='bwr')

                ax = plt.gca()

                cbar = plt.colorbar(im,fraction=0.0242, pad=0.01, format=ticker.FuncFormatter(fmt))
                cbar.set_label('Absolute error (MEFL)')
                cbar.set_ticks([-4e2, 0, 4e2])

            if scale == 'rel':
                
                Z = np.log2(Zd / Zm)
                
                im = plt.imshow(np.flipud(Z), interpolation='nearest', vmin=-1, vmax=1, cmap='bwr')

                ax = plt.gca()

                cbar = plt.colorbar(im,fraction=0.0242, pad=0.01, format=ticker.FuncFormatter(fmt))
                cbar.set_label('Log$_{2}$ relative error')
                cbar.set_ticks([-1, 0, 1])

            plt.xlabel("LED centroid wavelength (nm)")
            ax.set_xticks([i for i in range(len(pr.led_centroids))])
            ax.set_xticklabels(['{:.0f}'.format(i) for i in pr.led_centroids],rotation=60)

            round_to_n = lambda x, n: np.round(x, -int(np.floor(np.log10(x))) + (n - 1)) 
            plt.ylabel(int_label)
            ax.set_yticks(range(len(Y[:,0])))
            ax.set_yticklabels(['{}'.format(round_to_n(y, 2)) for y  in Y[::-1,0]])
    #         plt.ylabel(int_label)
    #         inten_ticks = [10, 1.0, 0.1, 0.01]
    #         def scale_inten(inten):
    #             return 1-np.log2(inten/10.0)
    #         ax.set_yticks([scale_inten(i) for i in inten_ticks])
    #         intensity_labels = ['{}'.format(i) for i in inten_ticks]
    #         ax.set_yticklabels(intensity_labels)

            ax = plt.gca()
            ax.minorticks_on()
            ax.tick_params(axis='x',which='both',top='off')
            ax.tick_params(axis='x',which='both',bottom='off')
            ax.tick_params(axis='y',which='both',left='off')
            ax.tick_params(axis='y',which='both',right='off')

            plt.tight_layout()

            plot_name = '{}_{}_{}_heatmap.png'.format(dset, scale, model)
            plt.savefig(out_folder + plot_name)
    #         plt.rcParams['savefig.dpi'] = 100
            plt.show()
            plt.close()

## Simulate spectral data for LED-by-LED plots

In [ ]:
df = dfs['aas'].sort_values(['Spectral intensity','Top Centroid (nm)'])
X = pr.led_centroids
Y = np.reshape(df['Spectral intensity'], (12,24))[:,0]
Y = np.logspace(np.log10(Y[0]), np.log10(Y[-1]), 100)
Z = np.reshape(df['FL1 Mean'], (12,24))[:,:23]
Z[-3:,0] = np.nan

m = lsc.Model.from_path('../cph8-ompr_model_parameters/rdv2_m01/all_data_onetau_with_kholds_and_adjusted_initial.params', hill_type='rep')
m.params['b'].value = m.params['b'].value - white

aas_progs = [[0 for x in range(100)] for y in range(Z.shape[1])]
ras_progs = [[0 for x in range(100)] for y in range(Z.shape[1])]
led_names = pr.led_names
for i,x in enumerate(X):
    for j,y in enumerate(Y):
        l1 = lsc.LEDProgram(led_names[i].lower(), y)
        aas_progs[i][j] = l1
        l2 = lsc.LEDProgram('660-ls', 1.25)
        ras_progs[i][j] = lsc.WellProgram.fromlist([l1, l2])
        
sims['aas'] = np.array([[m.simulate_led(aas_progs[i][j]) for i in range(len(aas_progs))] for j in range(len(aas_progs[0]))]).T
sims['ras'] = np.array([[m.simulate_well(ras_progs[i][j]) for i in range(len(ras_progs))] for j in range(len(ras_progs[0]))]).T

## Spectral LED-by-LED plots (linear axis)

In [ ]:
scales = ['lin']

plt.rcParams['savefig.dpi'] = 600
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.color_cycle'] = led_cycle
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['legend.loc'] = 'upper center'
plt.rcParams['legend.fontsize'] = plt.rcParams['font.size'] * 0.6
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handletextpad'] = 0.3
plt.rcParams['legend.borderaxespad'] = 0.5

plt.rcParams['font.size'] = 12
plt.rcParams['xtick.labelsize'] = plt.rcParams['font.size']
plt.rcParams['ytick.labelsize'] = plt.rcParams['font.size']

fig = plt.figure()

ax = fig.add_subplot(111)
ax.set_xlabel(int_label)
ax.set_ylabel(gfp_label)
ax.spines['top'].set_color('none')
ax.spines['bottom'].set_color('none')
ax.spines['left'].set_color('none')
ax.spines['right'].set_color('none')
ax.tick_params(labelcolor='w', top='off', bottom='off', left='off', right='off')

df1 = dfs['aas'].sort_values(['Spectral intensity','Top Centroid (nm)'])
df2 = dfs['ras'].sort_values(['Spectral intensity','Top Centroid (nm)'])

for scale in scales:
    for i in range(len(pr.led_names)):
        ax = fig.add_subplot(4,6,i+1)
    #     Xd = aas['spec_i_act'][:,0]
    #     Yd1 = aas['gm'][:,i]
    #     Yd2 = das['gm'][:,i]

    #     Xm = np.logspace(-3, 2, 100)
    #     Ym1 = [steadystate(results_aas_das.params,inten,        0, led1=i, led2=7,pcs1=pcs1,pcs2=pcs2) for inten in Xm]
    #     Ym2 = [steadystate(results_aas_das.params,inten,das_inten, led1=i, led2=7,pcs1=pcs1,pcs2=pcs2) for inten in Xm]


    #             X = np.reshape(df['Top Centroid (nm)'], (12,24))[:,:23]
    #             Y = np.reshape(df['Spectral intensity'], (12,24))[:,:23]

    #             if model in ['raw']:
    #                 Z = np.reshape(df['FL1 Mean'], (12,24))[:,:23]
    #                 Z[-3:,0] = np.nan
    #             if model in ['model']:
    #                 Z = sims[dset]
        
        if i == 0:
            imax = 9
        else:
            imax = 13
        
        Xd1 = np.reshape(df1['Spectral intensity'], (12,24))[:imax,i]
        Xd2 = np.reshape(df2['Spectral intensity'], (4, 24))[:imax-8,i]
        Yd1 = np.reshape(df1['FL1 Mean'], (12,24))[:imax,i]
        Yd2 = np.reshape(df2['FL1 Mean'], (4,24))[:imax-8,i]
        
        Xm = np.reshape(df1['Spectral intensity'], (12,24))[:,i]
        Xm = np.logspace(np.log10(Xm[0]), np.log10(Xm[-1]), 100)
        Ym1 = sims['aas']
        Ym2 = sims['ras']
        
        if scale in ['lin']:
            ax.semilogx(Xd1, Yd1, '.', color = pr.led_colors[i])
            ax.semilogx(Xd2, Yd2, '.', color = pr.led_colors[i], marker='s', ms=6, markeredgecolor='none')
            ax.semilogx(Xm, Ym1[i], '-', color = pr.led_colors[i])
            ax.semilogx(Xm, Ym2[i], '-', color = pr.led_colors[i])
            ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
            ax.set_ylim(gfp_range)
            ax.set_yticks(gfp_ticks)
        
        if scale in ['log']:
            ax.loglog(Xd1, Yd1, '.', color = pr.led_colors[i])
            ax.loglog(Xd2, Yd2, '.', color = pr.led_colors[i], marker='s', ms=6, markeredgecolor='none')
            ax.loglog(Xm, Ym1[i], '-', color = pr.led_colors[i])
            ax.loglog(Xm, Ym2[i], '-', color = pr.led_colors[i])
            ax.set_ylim(plt.ylim([3e1,3.0e5])) 
            
        
        ax.set_xlim(int_range_log)
        ax.set_xticks(int_ticks_log)

        ax = plt.gca()
        ax.minorticks_on()
        ax.tick_params(axis='x',which='both',top='off')
        ax.tick_params(axis='y',which='both',right='off')

        ax = plt.gca()
        ax.text(0.5,0.95,\
                r'{:.0f} nm'.format(pr.led_centroids[i]),\
                horizontalalignment='center',\
                verticalalignment='top',\
                fontsize = plt.rcParams['font.size'],
                transform=ax.transAxes)

    plt.tight_layout()
    plot_name = 'led_fit_{}.png'.format(scale)
    plt.savefig(out_folder + plot_name)
    plt.rcParams['savefig.dpi'] = 100
    plt.show()
    plt.close()

## Spectral LED-by-LED plots (log axis)

In [ ]:
scales = ['log']

plt.rcParams['savefig.dpi'] = 600
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.color_cycle'] = led_cycle
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['legend.loc'] = 'upper center'
plt.rcParams['legend.fontsize'] = plt.rcParams['font.size'] * 0.6
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handletextpad'] = 0.3
plt.rcParams['legend.borderaxespad'] = 0.5

plt.rcParams['font.size'] = 12
plt.rcParams['xtick.labelsize'] = plt.rcParams['font.size']
plt.rcParams['ytick.labelsize'] = plt.rcParams['font.size']

fig = plt.figure()

ax = fig.add_subplot(111)
ax.set_xlabel(int_label)
ax.set_ylabel(gfp_label)
ax.spines['top'].set_color('none')
ax.spines['bottom'].set_color('none')
ax.spines['left'].set_color('none')
ax.spines['right'].set_color('none')
ax.tick_params(labelcolor='w', top='off', bottom='off', left='off', right='off')

df1 = dfs['aas'].sort_values(['Spectral intensity','Top Centroid (nm)'])
df2 = dfs['ras'].sort_values(['Spectral intensity','Top Centroid (nm)'])

for scale in scales:
    for i in range(len(pr.led_names)):
        ax = fig.add_subplot(4,6,i+1)
    #     Xd = aas['spec_i_act'][:,0]
    #     Yd1 = aas['gm'][:,i]
    #     Yd2 = das['gm'][:,i]

    #     Xm = np.logspace(-3, 2, 100)
    #     Ym1 = [steadystate(results_aas_das.params,inten,        0, led1=i, led2=7,pcs1=pcs1,pcs2=pcs2) for inten in Xm]
    #     Ym2 = [steadystate(results_aas_das.params,inten,das_inten, led1=i, led2=7,pcs1=pcs1,pcs2=pcs2) for inten in Xm]


    #             X = np.reshape(df['Top Centroid (nm)'], (12,24))[:,:23]
    #             Y = np.reshape(df['Spectral intensity'], (12,24))[:,:23]

    #             if model in ['raw']:
    #                 Z = np.reshape(df['FL1 Mean'], (12,24))[:,:23]
    #                 Z[-3:,0] = np.nan
    #             if model in ['model']:
    #                 Z = sims[dset]

        if i == 0:
            imax = 8
        else:
            imax = 12
        
        Xd1 = np.reshape(df1['Spectral intensity'], (12,24))[:imax,i]
        Xd2 = np.reshape(df2['Spectral intensity'], (4, 24))[:imax-8,i]
        Yd1 = np.reshape(df1['FL1 Mean'], (12,24))[:imax,i]
        Yd2 = np.reshape(df2['FL1 Mean'], (4,24))[:imax-8,i]
        
        Xm = np.reshape(df1['Spectral intensity'], (12,24))[:,i]
        Xm = np.logspace(np.log10(Xm[0]), np.log10(Xm[-1]), 100)
        Ym1 = sims['aas']
        Ym2 = sims['ras']
        
        if scale in ['lin']:
            ax.semilogx(Xd1, Yd1, '.', color = pr.led_colors[i])
            ax.semilogx(Xd2, Yd2, '.', color = pr.led_colors[i], marker='s', ms=6, markeredgecolor='none')
            ax.semilogx(Xm, Ym1[i], '-', color = pr.led_colors[i])
            ax.semilogx(Xm, Ym2[i], '-', color = pr.led_colors[i])
            ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
            ax.set_ylim(gfp_range)
            ax.set_yticks(gfp_ticks)
        
        if scale in ['log']:
            ax.loglog(Xd1, Yd1, '.', color = pr.led_colors[i])
            ax.loglog(Xd2, Yd2, '.', color = pr.led_colors[i], marker='s', ms=6, markeredgecolor='none')
            ax.loglog(Xm, Ym1[i], '-', color = pr.led_colors[i])
            ax.loglog(Xm, Ym2[i], '-', color = pr.led_colors[i])
            ax.set_ylim(plt.ylim([3e1,1e4])) 
            
        
        ax.set_xlim(int_range_log)
        ax.set_xticks(int_ticks_log)

        ax = plt.gca()
        ax.minorticks_on()
        ax.tick_params(axis='x',which='both',top='off')
        ax.tick_params(axis='y',which='both',right='off')

        ax = plt.gca()
        ax.text(0.5,0.95,\
                r'{:.0f} nm'.format(pr.led_centroids[i]),\
                horizontalalignment='center',\
                verticalalignment='top',\
                fontsize = plt.rcParams['font.size'],
                transform=ax.transAxes)

    plt.tight_layout()
    plot_name = 'led_fit_{}.png'.format(scale)
    plt.savefig(out_folder + plot_name)
    plt.rcParams['savefig.dpi'] = 100
    plt.show()
    plt.close()

## Plot raw photoconversion rates

In [ ]:
plot_name = 'pcs.png'

pr.init_style()

plt.rcParams['savefig.dpi'] = 600
plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['font.size'] = 16
plt.rcParams['legend.loc'] = 'upper center'
plt.rcParams['legend.fontsize'] = plt.rcParams['font.size'] * 0.6
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handletextpad'] = 0.3
plt.rcParams['legend.borderaxespad'] = 0.5

m = lsc.Model.from_path('../cph8-ompr_model_parameters/rdv2_m01/all_data_onetau_with_kholds_and_adjusted_initial.params', hill_type='rep')

model = m.params

pcs1 = [model['k1_{}'.format(i)].value for i in range(len(pr.led_names))]
pcs2 = [model['k2_{}'.format(i)].value for i in range(len(pr.led_names))]

pcs1e = [model['k1_{}'.format(i)].stderr for i in range(len(pr.led_names))]
pcs2e = [model['k2_{}'.format(i)].stderr for i in range(len(pr.led_names))]

Xd = pr.led_centroids
Yd1 = pcs1
Ye1 = pcs1e
Yd2 = pcs2
Ye2 = pcs2e

plt.errorbar(Xd,Yd1,yerr=Ye1,fmt='-', color=pr.red_cycle[4], label='k1')
plt.errorbar(Xd,Yd2,yerr=Ye2,fmt='-', color=pr.red_cycle[7], label='k2')

plt.legend(loc='upper right')
ax = plt.gca()
# ax.get_legend().get_title().set_fontsize(plt.rcParams['font.size'] * 0.8)
    
plt.xlim([300,1000])
plt.xlabel("LED centroid wavelength (nm)")
plt.xticks([400,500,600,700,800,900])

plt.ylim([0,0.06])
plt.ylabel(r'Photoconversion rate ($\frac{\si{min^{-1}}}{\si{\micro\mole.m^{-2}.s^{-1}}}$)')
plt.yticks([0, 0.02, 0.04, 0.06])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

ax = plt.gca()
ax.minorticks_on()
ax.tick_params(axis='x',which='both',top='off')
# ax.tick_params(axis='x',which='both',bottom='off')
ax.tick_params(axis='y',which='both',right='off')
# ax.set_aspect(1./ax.get_data_ratio())
plt.tight_layout()

plt.savefig(out_folder + plot_name)
plt.rcParams['savefig.dpi'] = 600
plt.show()
plt.close()

## Plot multicollinearity results

In [ ]:
pr.init_style()

plt.rcParams['axes.color_cycle'] = green_cycle
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['figure.figsize'] = (4.4, 4)
plt.rcParams['legend.loc'] = 'upper center'
plt.rcParams['legend.fontsize'] = plt.rcParams['font.size'] * 0.6
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handletextpad'] = 0.3
plt.rcParams['legend.borderaxespad'] = 0.5

import pickle

# fname1 = 'models/data_plots/all_data_with_k_holds_ratio_k_fixed_onetau_'
# E:\OneDrive\Projects\12.LightSensorCharacterization\models\data_plots_and_fitting\rdv2\wholepop\fit_results
fname2 = './fit_results/all_data_with_few_k_holds_ratio_k_fixed_'
# fname3 = 'models/data_plots/all_data_with_fewer_k_holds_ratio_k_fixed_twotau_'

fnames = [ fname2,]

fnames = [os.path.abspath(x) for x in fnames]

kvals = np.logspace(-2,0,10).tolist()
values = [[],[],[]]
for kval in kvals:
    rs = [pickle.load(open(fn+'{}.p'.format(kval),'rb')) for fn in fnames]
    
    tokens = [
        ['rchisq', 'reduced chi-square ='],
        ['k', 'k:'],
        ['k1', 'k1_8:'],
        ['k2', 'k2_8:'],
        ]
    
    kis = [[r.find(t)+len(t) for t in zip(*tokens)[1]] for r in rs]

    for i in range(len(fnames)):
        values[i].append([float(rs[i][ki:ki+20].strip().split(' ')[0].split('\n')[0]) for ki in kis[i]])
        
d = [{}]
for j in range(len(fnames)):
    for i,t in enumerate(zip(*tokens)[0]):
        d[j][t] = zip(*values[j])[i]      
        
x ='k'
y ='rchisq'    
plt.semilogx(d[0][x], d[0][y]/np.min(d[0][y]), '.')
plt.ylim([0.95,1.15])
plt.xlim([0.009,1.1])
plt.xlabel("k")
plt.ylabel("Relative residual chi-squared")
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax = plt.gca()
ax.set_aspect(1./ax.get_data_ratio())
plt.tight_layout()
plt.savefig(out_folder+plot_name+'_chisq.png')
plt.show()
plt.close()

plt.rcParams['figure.figsize'] = (4, 4)
x ='k'
y ='k1'     
plt.semilogx(d[0][x], d[0][y], '.')
plt.ylim([0,5e-3])
plt.xlim([0.009,1.1])
plt.xlabel("k")
plt.ylabel("k_{1,14}")
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax = plt.gca()
ax.set_aspect(1./ax.get_data_ratio())
plt.tight_layout()
plt.savefig(out_folder+plot_name+'_k1.png')
plt.show()
plt.close()

x ='k'
y ='k2'     
plt.semilogx(d[0][x], d[0][y], '.')
plt.ylim([0,4e-3])
plt.xlim([0.009,1.1])
plt.yticks([0,1e-3,2e-3,3e-3,4e-3])
plt.xlabel("k")
plt.ylabel("k_{2,14}")
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
ax = plt.gca()
ax.set_aspect(1./ax.get_data_ratio())
plt.tight_layout()
plt.savefig(out_folder+plot_name+'_k2.png')
plt.show()
plt.close()

x ='k1'
y ='k2'    
plt.plot(d[0][x], d[0][y], '.')
plt.ylim([0,4e-3])
plt.xlim([0,5e-3])
plt.yticks([0,1e-3,2e-3,3e-3,4e-3])
plt.xlabel("k_{1,14}")
plt.ylabel("k_{2,14}")
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
# plt.xticks([0.002,0.008,0.014,0.02])
ax = plt.gca()
# ax.set_aspect(1./ax.get_data_ratio())
plt.tight_layout()
plt.savefig(out_folder+plot_name+'_k1k2.png')
plt.show()
plt.close()

# Dynamic validation results

## Preamble

In [ ]:
def genprog(modelid, refid, ledid, prefid=None, prefledid=None):
    df1 = genprog_df.loc[genprog_df['Model ID'] == modelid]
    df2 = df1.loc[df1['Ref ID'] == refid]
    df = df2.loc[df2['LED ID'] == ledid]
    if prefid:
        df = df.loc[df['Compensated perturbation ref ID'] == prefid]
        df = df.loc[df['Compensated perturbation LED ID'] == prefledid]

    genprog_times = df['Times'].iloc[0].strip('[]').split(',')
    genprog_times = np.array([float(t) for t in genprog_times])
    genprog_ints = df['Intensities'].iloc[0].strip('[]').split(',')
    genprog_ints = np.array([float(y) for y in genprog_ints])

    return genprog_times, genprog_ints

## Simulate dynamic validation experiments using fitted model

In [ ]:
datasets = ['dv-mono-uv', 'dv-mono-r', 'dv-pert', 'dv-comp']

modelid = 'rdv2_m01'
refid = 'ref_prog05'

ref = zip(ref_df['ref_prog05_x'].dropna(), ref_df['ref_prog05_y'].dropna())
pref = zip(ref_df['ref_pert01_x'].dropna(), ref_df['ref_pert01_y'].dropna())

m = lsc.Model.from_path('../cph8-ompr_model_parameters/rdv2_m01/all_data_onetau_with_kholds_and_adjusted_initial.params', hill_type='rep')
m.params['b'].value = m.params['b'].value - white

for dataset in datasets:
    Ys_all = []
    
    if dataset in ['dv-mono-r', 'dv-pert', 'dv-comp']:
        ledid = '660-ls'
    elif dataset in ['dv-mono-uv']:
        ledid = '380-sb'
    
    Xs = np.linspace(0,480,481)
    if dataset in ['dv-mono-uv','dv-mono-r', 'dv-pert']:
        Xlp1, Ylp1 = genprog(modelid, refid, ledid)
        l1 = lsc.LEDProgram.fromdict({'id': ledid, 'pre': 20, 'x': Xlp1, 'y': Ylp1}) 
    elif dataset in ['dv-comp']:
        Xlp1, Ylp1 = genprog(modelid, refid, ledid, 'ref_pert01', '520-2-kb')
        l1 = lsc.LEDProgram.fromdict({'id': ledid, 'pre': 20, 'x': Xlp1, 'y': Ylp1})

    if dataset in ['dv-mono-uv', 'dv-mono-r']:
        l2 = lsc.LEDProgram()

    if dataset in ['dv-pert', 'dv-comp']:
        prefledid = '520-2-kb'
        Xrp, Yrp = zip(*pref)
        l2 = lsc.LEDProgram.fromdict({'id': prefledid, 'pre': 0, 'x': Xrp, 'y': Yrp})
    
    wp = lsc.WellProgram.fromlist([l1,l2])
    
    Ysingle = m.simulate_well(wp)(Xs)
    np.savetxt(out_folder + 'sim_{}_best.csv'.format(dataset), Ysingle, delimiter = ',')   

## Plot dynamic validation data and calculate RMSEs

In [ ]:
datasets = ['dv-mono-uv', 'dv-mono-r', 'dv-pert', 'dv-comp']
# datasets = ['dv-pert']
scales = ['logy']
models = ['full']

pr.init_style()

plt.rcParams['figure.figsize'] = (4, 6)
plt.rcParams['axes.color_cycle'] = pr.red_cycle
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['legend.loc'] = 'upper center'
plt.rcParams['legend.fontsize'] = plt.rcParams['font.size'] * 0.6
plt.rcParams['legend.numpoints'] = 1
plt.rcParams['legend.handletextpad'] = 0.3
plt.rcParams['legend.borderaxespad'] = 0.5

ref = zip(ref_df['ref_prog05_x'].dropna(), ref_df['ref_prog05_y'].dropna())
pref = zip(ref_df['ref_pert01_x'].dropna(), ref_df['ref_pert01_y'].dropna())

m = lsc.Model.from_path('../cph8-ompr_model_parameters/rdv2_m01/all_data_onetau_with_kholds_and_adjusted_initial.params', hill_type='rep')
m.params['b'].value = m.params['b'].value - white

a, b = m.params['a'], m.params['b']

for dset in datasets:
    for model in models:
        for scale in scales:
            
            df = dfs[dset].sort_values(['Data time point'])
        
            Xd = []
            Yd = []
            Yde = []

            t_set = set(df['Data time point'].values)
            t_set = sorted(t_set)
            for t in t_set:
                dft = df.loc[df['Data time point'] == t]
                dft = dft.dropna(subset=['FL1 Mean'])
                Xd.append(t)
                Yd.append(np.mean(dft['FL1 Mean'].values))
                Yde.append(np.std(dft['FL1 Mean'].values))
            
            Xr, Yr = zip(*ref)
            Yr = np.array(Yr)
            
            Yr = b + a*Yr
#             Yr = b + a*Yr_b
            
#             print Yr
            
            modelid = 'rdv2_m01'
            refid = 'ref_prog05'
            
            if dset in ['dv-mono-r', 'dv-pert', 'dv-comp']:
                ledid = '660-ls'
                lpcolor1 = pr.led_colors[14]
            elif dset in ['dv-mono-uv']:
                ledid = '380-sb'
                lpcolor1 = pr.led_colors[1]
            
            if dset in ['dv-pert', 'dv-comp']:
                prefledid = '520-2-kb'
                Xrp, Yrp = zip(*pref)
                lpcolor2 = pr.led_colors[8]
                
#             Xs = np.linspace(0,480,481)
            if dset in ['dv-mono-uv','dv-mono-r', 'dv-pert']:
                Xlp1, Ylp1 = genprog(modelid, refid, ledid)
                l1 = lsc.LEDProgram.fromdict({'id': ledid, 'pre': 20, 'x': Xlp1, 'y': Ylp1}) 
            elif dset in ['dv-comp']:
                Xlp1, Ylp1 = genprog(modelid, refid, ledid, 'ref_pert01', '520-2-kb')
                l1 = lsc.LEDProgram.fromdict({'id': ledid, 'pre': 20, 'x': Xlp1, 'y': Ylp1})
                
#             if dset in ['dv-mono-uv', 'dv-mono-g']:
#                 Ys = m.simulate_led(l1)(Xs)
                
            if dset in ['dv-pert', 'dv-comp']:
                l2 = lsc.LEDProgram.fromdict({'id': prefledid, 'pre': 0, 'x': Xrp, 'y': Yrp})
                wp = lsc.WellProgram.fromlist([l1,l2])
#                 Ys = m.simulate_well(wp)(Xs)
            
            Ysingle = np.loadtxt(out_folder + 'sim_{}_best.csv'.format(dset))
            Ysingle = interp1d(Xs, Ysingle)
            
            errs = Yd - Ysingle(Xd)
            sumsq = np.sqrt(np.sum(errs**2)/len(errs))
            print '{} sim RMSE (MEFL): {}'.format(dset, sumsq)
            print '{} sim NRMSE: {}'.format(dset, 100*sumsq/a)
            
            errs = (Yd - Ysingle(Xd))/Yd
            sumsq = np.sqrt(np.sum(errs**2)/len(errs))
            print '{} sim RMSE (Relative): {}'.format(dset, 100*sumsq)
            
            errs = np.log10(Yd/Ysingle(Xd))
            sumsq = np.sqrt(np.sum(errs**2)/len(errs))
            print '{} sim RMSE (Log10): {}'.format(dset, sumsq)
            print '{} sim RMSE (Log10->Lin): {}'.format(dset, 100*(10**sumsq-1))
            
            errs = np.mean(Yde)
            print '{} errorbar avg (MEFL): {}'.format(dset, errs)
            
            errs = np.mean(np.array(Yde)/np.array(Yd))
            print '{} errorbar CV: {}'.format(dset, 100*errs)
            
            errs = np.mean((Ysingle(Xd) - Yd) / Yde)
            print '{} sigma avg: {}'.format(dset, errs)
            
            errs = np.sqrt(np.mean( ((Ysingle(Xd) - Yd) / Yde)**2 ))
            print '{} sigma RMSE: {}'.format(dset, errs)
            
            # Data plot
            plt.figure(figsize = (2.05*1.5/1.141, 2.05*1.5/1.141))
#             plt.fill_between(Xs, Ys[0,:], Ys[1,:], color='gray', alpha=0.6, linewidth=0, label='Sim', zorder=2)
            plt.plot(Xs,Ysingle(Xs),'-',color=pr.green_cycle[3],lw=2.5,zorder=1,alpha=0.7)
            
            plt.plot(Xr, Yr, 'k-', zorder=2, lw=0.75)
            plt.errorbar(Xd, Yd, Yde, color=pr.green_cycle[3], fmt='.', label='Data',ms=6, elinewidth=1, capsize=1.5, capthick=1)
            
            ax1 = plt.gca()
            ax1.set_xlim([0,480])
            ax1.set_xticks([0,120,240,360,480])
            ax1.set_xticklabels([])
            
            ax1.set_yscale('log')
            ax1.set_ylim([3e1, 1e4])
            ax1.set_yticks([1e2,1e3,1e4])
            ax1.set_yticklabels([])

            ax1.tick_params(axis='x',which='both',top='off')
            ax1.tick_params(axis='y',which='both',right='off')
            
#             plt.tight_layout()
            plot_name = '{}_{}_{}_data.png'.format(dset, scale, model)
            plt.savefig(out_folder + plot_name, pad_inches=0, bbox_inches='tight', transparent=True)
            plt.show()
            plt.close()
            plt.clf()
            
            # Linear light plot
            plt.figure(figsize = (2.05*1.5/1.141, 0.751*1.5/1.141))
            plt.step(Xlp1, Ylp1, color=lpcolor1)
            if dset in ['dv-pert', 'dv-comp']:
                plt.step(Xrp, Yrp, color=lpcolor2)    
            
            ax2 = plt.gca()
            ax2.set_xlim([0,480])
            ax2.set_xticks([0,120,240,360,480])
            ax2.set_xticklabels([])
            
            ax2.set_ylim([0,25])
            ax2.set_yticks([0,10,20])
            ax2.set_yticklabels([])
            
            ax2.tick_params(axis='x',which='both',top='off')
            ax2.tick_params(axis='y',which='both',right='off')
            
#             plt.tight_layout()
            plot_name = '{}_{}_{}_intlin.png'.format(dset, scale, model)
            plt.savefig(out_folder + plot_name, pad_inches=0, bbox_inches='tight', transparent=True)
            plt.show()
            plt.close()
            plt.clf()
            
            # Log light plot
            plt.figure(figsize = (2.05*1.5/1.141, 0.751*1.5/1.141))
            plt.step(Xlp1, Ylp1, color=lpcolor1)
            if dset in ['dv-pert', 'dv-comp']:
                plt.step(Xrp, Yrp, color=lpcolor2)    
            
            ax3 = plt.gca()
            ax3.set_xlim([0,480])
            ax3.set_xticks([0,120,240,360,480])
            ax3.set_xticklabels([])
            
            ax3.set_yscale('log')
            ax3.set_ylim([2e-2,4e1])
            ax3.set_yticks([1e-1, 1e0, 1e1])
            ax3.set_yticklabels([])
            
            ax3.tick_params(axis='x',which='both',top='off')
            ax3.tick_params(axis='y',which='both',right='off')

#             plt.tight_layout()
            plot_name = '{}_{}_{}_intlog.png'.format(dset, scale, model)
            plt.savefig(out_folder + plot_name, pad_inches=0, bbox_inches='tight', transparent=True)
            plt.show()
            plt.close()
            plt.clf()